# Intro

This is my first time working on a Question-Answering system in NLP. Most of this code was obtained by referencing the work of [pallavrajsahoo]((https://github.com/pallavrajsahoo/Question-Answering-System-with-SQuAD-dataset)). A big shoutout to him for helping me understand what to do with this dataset! It's possible that by the time you're reading my notebook, it won't be perfectly finished. I'll do my best to keep updating the notebook until it's complete

## Brief Description About the Datasest

**Stanford Question Answering Dataset (SQuAD)** is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

1. [tf-link](https://www.tensorflow.org/datasets/catalog/squad) 
2. [officialHomepage](https://rajpurkar.github.io/SQuAD-explorer/) 
3. [paperDoc](https://paperswithcode.com/dataset/squad)



In [1]:
import pandas as pd
import numpy as np
import torch
import os, sys, random, pickle, json
from datasets import load_dataset, list_datasets

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download("stopwords", quiet = True)
nltk.download("wordnet", quiet = True)
nltk.download("averaged_perceptron_tagger", quiet = True)
nltk.download("punkt", quiet = True)


/home/abhinowo/Documents/LearningProject/hfSeries/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
!nvidia-smi

Fri Oct 27 14:13:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P5               6W /  50W |    674MiB /  8188MiB |     21%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
device = torch.device('cuda' if torch .cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
dataset = list_datasets()
len(dataset)

/tmp/ipykernel_29371/3844060883.py:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  dataset = list_datasets()


73461

In [5]:
GLOBAL_SEED = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(GLOBAL_SEED)

In [6]:
def check_equal(actual, expected):
    assert actual == expected, actual

def check_approx(actual, expected):
    assert np.allclose(actual, expected), actual

- dont mind this **squad_v2**. ill try from squad dataset first 

In [7]:
squad_dataset2 = load_dataset("squad_v2")
squad_dataset2

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [8]:
df_2 = pd.DataFrame(squad_dataset2["train"])
df_2.head()

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"


In [9]:
df_2.iloc[0].to_dict()

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

------------------------------------------------------------------------------------

In [10]:
squad_dataset = load_dataset('squad')
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [11]:
squad_dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [12]:
# turn the dataset into a dataframe

df = pd.DataFrame(squad_dataset['train'])
df.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [13]:
# clean answers column

df['answer_start'] = df['answers'].apply(lambda x: x['answer_start'][0])
df['answers'] = df['answers'].apply(lambda x: x['text'][0])
df.head()

,id,title,context,question,answers,answer_start
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92


In [14]:
df['title'].value_counts().sum()

87599

In [15]:
df['context'].loc[0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [16]:
# drop id column

df.drop('id', axis = 1, inplace = True)
df.head()

,title,context,question,answers,answer_start
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         87599 non-null  object
 1   context       87599 non-null  object
 2   question      87599 non-null  object
 3   answers       87599 non-null  object
 4   answer_start  87599 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.3+ MB


In [18]:
# turn title to string, context to string, question to string, answers to string and answer_start to int

df['title'] = df['title'].astype(str)
df['context'] = df['context'].astype(str)
df['question'] = df['question'].astype(str)
df['answers'] = df['answers'].astype(str)
df['answer_start'] = df['answer_start'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         87599 non-null  object
 1   context       87599 non-null  object
 2   question      87599 non-null  object
 3   answers       87599 non-null  object
 4   answer_start  87599 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.3+ MB


In [19]:
df.iloc[0].to_dict()

{'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': 'Saint Bernadette Soubirous',
 'answer_start': 515}

## Unsupervised Models

In [20]:
# trying check similarity between two sentences - test

def jaccard_similarity(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

jaccard_similarity("My name is John", "John is my name")


1.0

In [21]:
# implement jaccard similarity between question and answer

def jaccard_similarity(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

df['jaccard_similarity'] = df.apply(lambda x: jaccard_similarity(x['question'], x['answers']), axis = 1)
df.head()


,title,context,question,answers,answer_start,jaccard_similarity
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000


In [22]:
df.sort_values(by= 'jaccard_similarity',ascending = False).head(10)

,title,context,question,answers,answer_start,jaccard_similarity
16820,Nanjing,"Nanjing ( listen; Chinese: 南京, ""Southern Capit...",n,n,2,1.000000
16821,Nanjing,"Nanjing ( listen; Chinese: 南京, ""Southern Capit...",b,b,144,1.000000
16822,Nanjing,"Nanjing ( listen; Chinese: 南京, ""Southern Capit...",v,v,108,1.000000
30525,Federal_Bureau_of_Investigation,"In response to organized crime, on August 25, ...",Is the RICO Act still used today?,The RICO Act is still used,939,0.857143
55172,FA_Cup,"Following the 1914–15 edition, the competition...",Was competition suspended due to the first wor...,competition was suspended due to the First Wor...,35,0.800000
14006,Internet_service_provider,"Network hardware, software and specifications,...",Is a tradeoff between efficiency and cost poss...,A tradeoff between cost and efficiency is poss...,214,0.777778
48862,Somerset,Until the 1960s the piers at Weston-super-Mare...,What is the shortest pier in the UK,The pier at Burnham-on-Sea is the shortest pie...,417,0.666667
77599,Premier_League,Due to insistence by the International Federat...,What was the number of clubs reduced to in 1995.,the number of clubs was reduced to 20 in 1995,191,0.666667
55287,FA_Cup,Seven clubs have won the FA Cup as part of a L...,How many clubs have won the fa cup as part of ...,Seven clubs have won the FA Cup as part of a L...,0,0.647059
57999,Political_party,"In a nonpartisan system, no official political...",Do official political parties exist in a nonpa...,"In a nonpartisan system, no official political...",0,0.636364


In [23]:
# check why is there some weird value in answer and question

df.iloc[16820].to_dict()

{'title': 'Nanjing',
 'context': 'Nanjing ( listen; Chinese: 南京, "Southern Capital") is the city situated in the heartland of lower Yangtze River region in China, which has long been a major centre of culture, education, research, politics, economy, transport networks and tourism. It is the capital city of Jiangsu province of People\'s Republic of China and the second largest city in East China, with a total population of 8,216,100, and legally the capital of Republic of China which lost the mainland during the civil war. The city whose name means "Southern Capital" has a prominent place in Chinese history and culture, having served as the capitals of various Chinese dynasties, kingdoms and republican governments dating from the 3rd century AD to 1949. Prior to the advent of pinyin romanization, Nanjing\'s city name was spelled as Nanking or Nankin. Nanjing has a number of other names, and some historical names are now used as names of districts of the city, and among them there is the

In [24]:
df.sort_values(by= 'jaccard_similarity',ascending = True).head(10)

,title,context,question,answers,answer_start,jaccard_similarity
54923,Tuberculosis,A number of medications are being studied for ...,"Despite FDA approval, how much more likely to ...",five times,444,0.0
54934,Affirmative_action_in_the_United_States,Affirmative action is a subject of controversy...,What do critics of affirmative action believe ...,achievement,961,0.0
54933,Affirmative_action_in_the_United_States,Affirmative action is a subject of controversy...,In which time period did discrimination polici...,Reconstruction Era,557,0.0
54932,Affirmative_action_in_the_United_States,Affirmative action is a subject of controversy...,What case determined that some implementation ...,Gratz v. Bollinger,321,0.0
54931,Affirmative_action_in_the_United_States,Affirmative action is a subject of controversy...,Having quotas regarding admissions or employme...,reverse,192,0.0
54930,Affirmative_action_in_the_United_States,Affirmative action is a subject of controversy...,What is another example aside from racial quot...,gender quotas,118,0.0
54927,Affirmative_action_in_the_United_States,Affirmative action in the United States tends ...,Affirmative action attempts to ask institution...,racial minorities,146,0.0
54926,Affirmative_action_in_the_United_States,Affirmative action in the United States tends ...,"Outside of employment, what is the other main ...",education,73,0.0
54925,Tuberculosis,A number of medications are being studied for ...,What type of publication has put out articles ...,medical journal,529,0.0
54924,Tuberculosis,A number of medications are being studied for ...,Opposition to the use of bedaquiline think wha...,physicians,694,0.0


In [25]:
# lets see why it got such a low score

df.iloc[54923].to_dict()

{'title': 'Tuberculosis',
 'context': "A number of medications are being studied for multi drug resistant tuberculosis including: bedaquiline and delamanid. Bedaquiline received U.S. Food and Drug Administration (FDA) approval in late 2012. The safety and effectiveness of these new agents are still uncertain, because they are based on the results of a relatively small studies. However, existing data suggest that patients taking bedaquiline in addition to standard TB therapy are five times more likely to die than those without the new drug, which has resulted in medical journal articles raising health policy questions about why the FDA approved the drug and whether financial ties to the company making bedaquiline influenced physicians' support for its use ",
 'question': 'Despite FDA approval, how much more likely to die are patients who take bedaquiline in addition to the standard TB regimen?',
 'answers': 'five times',
 'answer_start': 444,
 'jaccard_similarity': 0.0}

In [26]:
# TF-IDF 

from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_similarity(str1, str2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([str1, str2])
    return ((tfidf_matrix * tfidf_matrix.T).A)[0,1]

tfidf_similarity("My name is John", "John is my name")

1.0

In [27]:
tfidf_vectorizer = TfidfVectorizer(
    tokenizer = nltk.word_tokenize,
    stop_words = stopwords.words('english'),
    # lowercase = True
    ngram_range = (1, 2),
    max_features = 10000,
)

tfidf_vectorizer.fit(df['context'].unique())

/home/abhinowo/Documents/LearningProject/hfSeries/.venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/abhinowo/Documents/LearningProject/hfSeries/.venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


TfidfVectorizer(max_features=10000, ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function word_tokenize at 0x7f06419fdb40>)

## load the validation data

In [28]:
# turn the dataset into a dataframe

df_val = pd.DataFrame(squad_dataset['validation'])
df_val.head()

,id,title,context,question,answers
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ..."
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth..."
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S..."
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ..."
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta..."


In [29]:
# do the same thing as we did for the train dataset

df_val['answer_start'] = df_val['answers'].apply(lambda x: x['answer_start'][0])
df_val['answers'] = df_val['answers'].apply(lambda x: x['text'][0])
df_val.drop('id', axis = 1, inplace = True)
df_val['title'] = df_val['title'].astype(str)
df_val['context'] = df_val['context'].astype(str)
df_val['question'] = df_val['question'].astype(str)
df_val['answers'] = df_val['answers'].astype(str)
df_val['answer_start'] = df_val['answer_start'].astype(int)
df_val.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10570 entries, 0 to 10569
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         10570 non-null  object
 1   context       10570 non-null  object
 2   question      10570 non-null  object
 3   answers       10570 non-null  object
 4   answer_start  10570 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 413.0+ KB


In [30]:
df_val

,title,context,question,answers,answer_start
0,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos,177
1,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers,249
2,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"Santa Clara, California",403
3,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,Denver Broncos,177
4,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,gold,488
...,...,...,...,...,...
10565,Force,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,kilogram-force,82
10566,Force,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,kilopond,114
10567,Force,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,slug,274
10568,Force,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,kip,712


In [31]:
# test tfidf vectorizer

test_context = df_val['context']
tfid_vector_val = tfidf_vectorizer.transform(test_context)

# find the most similar context to the question

def find_similar_question(question):
    tfid_vector_question = tfidf_vectorizer.transform([question])
    similarity = np.dot(tfid_vector_val, tfid_vector_question.T).toarray()
    return similarity.argmax()

find_similar_question("What is the name of the university?")
df_val['context'].loc[find_similar_question("What is the name of the university?")]



'The city has two universities — Newcastle University and Northumbria University. Newcastle University has its origins in the School of Medicine and Surgery, established in 1834 and became independent from Durham University on 1 August 1963 to form the University of Newcastle upon Tyne. Newcastle University is now one of the UK\'s leading international universities. It won the coveted Sunday Times University of the Year award in 2000. Northumbria University has its origins in the Newcastle Polytechnic, established in 1969 and became the University of Northumbria at Newcastle in 1992 as part of the UK-wide process in which polytechnics became new universities. Northumbria University was voted \'Best New University\' by The Times Good University Guide 2005 and also won a much coveted company award of the "Most IT enabled organisation" (in the UK), by the IT industry magazine Computing.'

In [34]:
# tokenization for bert

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

(…)cased/resolve/main/tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 216kB/s]
(…)bert-base-uncased/resolve/main/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 71.9MB/s]
(…)base-uncased/resolve/main/tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 34.2MB/s]
(…)rt-base-uncased/resolve/main/config.json: 100%|██████████| 570/570 [00:00<00:00, 5.99MB/s]


In [35]:
# tokenize the context

def tokenize_context(context):
    return tokenizer.tokenize(context)

df['tokenized_context'] = df['context'].apply(lambda x: tokenize_context(x))
df.head()


,title,context,question,answers,answer_start,jaccard_similarity,tokenized_context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000,"[architectural, ##ly, ,, the, school, has, a, ..."
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429,"[architectural, ##ly, ,, the, school, has, a, ..."
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667,"[architectural, ##ly, ,, the, school, has, a, ..."
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000,"[architectural, ##ly, ,, the, school, has, a, ..."
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000,"[architectural, ##ly, ,, the, school, has, a, ..."


In [36]:
# after tokenize the context, we need to tokenize the question

def tokenize_question(question):
    return tokenizer.tokenize(question)

df['tokenized_question'] = df['question'].apply(lambda x: tokenize_question(x))
df.head()

,title,context,question,answers,answer_start,jaccard_similarity,tokenized_context,tokenized_question
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[to, whom, did, the, virgin, mary, allegedly, ..."
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, in, front, of, the, notre, dame, ma..."
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667,"[architectural, ##ly, ,, the, school, has, a, ...","[the, basilica, of, the, sacred, heart, at, no..."
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, the, gr, ##otto, at, notre, dame, ?]"
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, sits, on, top, of, the, main, building,..."


In [37]:
# now we need to tokenize the answer

def tokenize_answer(answer):
    return tokenizer.tokenize(answer)

df['tokenized_answer'] = df['answers'].apply(lambda x: tokenize_answer(x))
df.head()

,title,context,question,answers,answer_start,jaccard_similarity,tokenized_context,tokenized_question,tokenized_answer
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[to, whom, did, the, virgin, mary, allegedly, ...","[saint, bern, ##ade, ##tte, so, ##ub, ##iro, #..."
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, in, front, of, the, notre, dame, ma...","[a, copper, statue, of, christ]"
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667,"[architectural, ##ly, ,, the, school, has, a, ...","[the, basilica, of, the, sacred, heart, at, no...","[the, main, building]"
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, the, gr, ##otto, at, notre, dame, ?]","[a, marian, place, of, prayer, and, reflection]"
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, sits, on, top, of, the, main, building,...","[a, golden, statue, of, the, virgin, mary]"


In [38]:
# we can see that the tokenized answer is not the same as the answer, so we need to find the answer in the tokenized context

def find_answer_in_tokenized_context(tokenized_context, tokenized_answer):
    for i in range(len(tokenized_context)):
        if tokenized_context[i:i+len(tokenized_answer)] == tokenized_answer:
            return i, i+len(tokenized_answer)
    return -1, -1

df['answer_start_tokenized'], df['answer_end_tokenized'] = zip(*df.apply(lambda x: find_answer_in_tokenized_context(x['tokenized_context'], x['tokenized_answer']), axis = 1))
df.head()


,title,context,question,answers,answer_start,jaccard_similarity,tokenized_context,tokenized_question,tokenized_answer,answer_start_tokenized,answer_end_tokenized
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[to, whom, did, the, virgin, mary, allegedly, ...","[saint, bern, ##ade, ##tte, so, ##ub, ##iro, #...",113,121
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, in, front, of, the, notre, dame, ma...","[a, copper, statue, of, christ]",39,44
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667,"[architectural, ##ly, ,, the, school, has, a, ...","[the, basilica, of, the, sacred, heart, at, no...","[the, main, building]",11,14
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, the, gr, ##otto, at, notre, dame, ?]","[a, marian, place, of, prayer, and, reflection]",84,91
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, sits, on, top, of, the, main, building,...","[a, golden, statue, of, the, virgin, mary]",19,26


In [40]:
# find the score prediction for the context and answer

def find_score_prediction(context, answer):
    return tfidf_similarity(context, answer)

df['score_prediction'] = df.apply(lambda x: find_score_prediction(x['context'], x['answers']), axis = 1)
df.head()



,title,context,question,answers,answer_start,jaccard_similarity,tokenized_context,tokenized_question,tokenized_answer,answer_start_tokenized,answer_end_tokenized,score,score_prediction
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[to, whom, did, the, virgin, mary, allegedly, ...","[saint, bern, ##ade, ##tte, so, ##ub, ##iro, #...",113,121,8,0.056337
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0.071429,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, in, front, of, the, notre, dame, ma...","[a, copper, statue, of, christ]",39,44,5,0.219739
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,0.066667,"[architectural, ##ly, ,, the, school, has, a, ...","[the, basilica, of, the, sacred, heart, at, no...","[the, main, building]",11,14,3,0.478603
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0.000000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, is, the, gr, ##otto, at, notre, dame, ?]","[a, marian, place, of, prayer, and, reflection]",84,91,7,0.221936
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,0.125000,"[architectural, ##ly, ,, the, school, has, a, ...","[what, sits, on, top, of, the, main, building,...","[a, golden, statue, of, the, virgin, mary]",19,26,7,0.514849
